In [6]:
%run pool_config_optimization_python_functions.ipynb

In [7]:
f = open('igs_10_16_pool_report.json') 
data = json.load(f)

In [8]:

base_pool_config = {'auto_shutdown': 10, 'min_idle_vm': 0}

auto_shutdown_grid = [1, 2, 3, 4, 5, 6, 10]
min_idle_vm_grid = [0, 1, 2, 3, 4, 5, 6, 7, 8]

if len(base_pool_config)>0:
    if base_pool_config['auto_shutdown'] not in auto_shutdown_grid:
        auto_shutdown_grid.append(base_pool_config['auto_shutdown'])
    if base_pool_config['min_idle_vm'] not in min_idle_vm_grid:
        min_idle_vm_grid.append(base_pool_config['min_idle_vm']) 

In [9]:
# creating clusters dataframe
clusters_pd = cluster_df(data)

# creating contentionTimelines dataframe
contentionTimelines_pd = contentionTimelines_df(data)

# merging two dataframes
cluster_contention_pd = contentionTimelines_pd.merge(clusters_pd, on='clusterId', how='left')

# filtering for only job clusters
cluster_contention_jc_pd=cluster_contention_pd[cluster_contention_pd.clusterType=='JOB_CLUSTER']

# creating cluster timeline dataframe
cluster_timelines_pd =  cluster_timeline_df(cluster_contention_jc_pd)

In [10]:
greedy_df = pd.DataFrame(columns=['auto_shutdown', 'min_idle_vm', 'total_idle_time',
                                  'idle_vm_count', 'cold_start_cnt', 'warm_start_cnt',])
cnt = 0
t1 = time.time()
for auto_shutdown in auto_shutdown_grid:

    # step 2
    cluster_groups = grouper(cluster_timelines_pd, auto_shutdown)


    # Adding group number to cluster_timelines_pd
    cluster_timelines_pd = cluster_timeline_update(cluster_timelines_pd, cluster_groups)


    # step 3
    total_groups_distance, cluster_groups = group_intervals(cluster_groups)

    # step 4, part B
    vm_usage_classes = vm_usage_v4(cluster_groups, cluster_timelines_pd, cluster_contention_jc_pd, auto_shutdown)


    # groups summary and stats
    groups_summary_org = groups_stats_v2(vm_usage_classes)

    # calculating total idle time in sec
    total_idle_time_org = groups_summary_org.idle_duration.sum()


    ## Part g: add effects of min_idle_vm
    for min_idle_vm in min_idle_vm_grid:
        total_idle_time, groups_summary = min_idle_vm_effect_v2(min_idle_vm, total_idle_time_org,
                                                             total_groups_distance, groups_summary_org)

        # creating greedy dataframe
        greedy_df.loc[cnt] = [None]*len(greedy_df.columns)
        greedy_df['auto_shutdown'].loc[cnt] = auto_shutdown
        greedy_df['min_idle_vm'].loc[cnt] = min_idle_vm
        greedy_df['total_idle_time'].loc[cnt] = total_idle_time
        
        greedy_df['idle_vm_count'].loc[cnt] = groups_summary.idle_vm_count.sum()
        greedy_df['cold_start_cnt'].loc[cnt] = groups_summary.cold_start_cnt.sum()
        greedy_df['warm_start_cnt'].loc[cnt] = groups_summary.warm_start_cnt.sum()
        cnt+=1


# finding optimal config       
opt_config = optimal_config(greedy_df)
t2 = time.time()
print(f'duration is {np.round(t2-t1,2)} seconds')
# calculating amount of improvements comapred to baseline usage
if len(base_pool_config)>0:
    percent_improvements = improvements(greedy_df, base_pool_config)
    print(f'The rate of improvements in using pool resources will be {percent_improvements}%')
    
print(f'The optimal suggested pool config is {opt_config}')        

duration is 8.42 seconds
The rate of improvements in using pool resources will be 42.4%
The optimal suggested pool config is {'auto_shutdown': 1, 'min_idle_vm': 0}
